In [1]:
from pyspark.sql import SparkSession
sparkSession = SparkSession.builder.enableHiveSupport().master("local[2]").getOrCreate()

In [2]:
data = sparkSession.read.parquet("/data/sample264")
meta = sparkSession.read.parquet("/data/meta")

In [3]:
from pyspark.sql import Window
from pyspark.sql.functions import *

def norm(df, key1, field, n): 
    
    window = Window.partitionBy(key1).orderBy(col(field).desc())
        
    topsDF = df.withColumn("row_number", row_number().over(window)) \
        .filter(col("row_number") <= n) \
        .drop(col("row_number")) 
        
    tmpDF = topsDF.groupBy(col(key1)).agg(col(key1), sum(col(field)).alias("sum_" + field))
   
    normalizedDF = topsDF.join(tmpDF, key1, "inner") \
        .withColumn("norm_" + field, col(field) / col("sum_" + field)) \
        .cache()

    return normalizedDF

In [6]:
data.cache()

data_copy = data.withColumnRenamed("timestamp", "timestamp2").withColumnRenamed("trackId", "trackId2")

track_to_track = data \
    .join(data_copy, "userId") \
    .filter(~(data.trackId==data_copy.trackId2) & (data_copy.timestamp2 >= data.timestamp)) \
    .withColumn("time_diff", col("timestamp2")-col("timestamp")) \
    .filter(col("time_diff") <= 7*60) \
    .withColumn("tr1", least(col("trackId"), col("trackId2"))) \
    .withColumn("tr2", greatest(col("trackId"), col("trackId2"))) \
    .groupBy(col("tr1"), col("tr2")).count()
   

normalized = norm(track_to_track, "tr1", "count", 40) \
    .withColumn("id", column("tr1")) \
    .withColumn("id2", column("tr2")) \
    .select(col("id"), col("id2"), col("norm_count"))     

window = Window.orderBy(col("norm_count").desc())

top = normalized.withColumn("pos", rank().over(window)) \
    .filter(col("pos") <= 40) \
    .orderBy(col("id").asc(), col("id2").asc()) \
    .select(col("id"), col("id2")) \
    .take(40)

    
for item in top:
    print "%s %s" % item
    
    




798256 923706
798319 837992
798322 876562
798331 827364
798335 840741
798374 816874
798375 810685
798379 812055
798380 840113
798396 817687
798398 926302
798405 867217
798443 905923
798457 918918
798460 891840
798461 940379
798470 840814
798474 963162
798477 883244
798485 955521
798505 905671
798545 949238
798550 936295
798626 845438
798691 818279
798692 898823
798702 811440
798704 937570
798725 933147
798738 894170
798745 799665
798782 956938
798801 950802
798820 890393
798833 916319
798865 962662
798931 893574
798946 946408
799012 809997
799024 935246
